<a href="https://colab.research.google.com/github/YoussefMkhaouad/Fece_recognition/blob/master/face_recognition_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Les bibliothèques

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import itertools

Using TensorFlow backend.


# Téléchargement de données

In [2]:
data = np.load('/content/drive/My Drive/ORL_faces.npz') 

x_train = data['trainX']

x_train = np.array(x_train,dtype='float32')/255

x_test = data['testX']
x_test = np.array(x_test,dtype='float32')/255


y_train= data['trainY']
y_test= data['testY']


print('x_train : {}'.format(x_train[:]))
print('Y-train shape: {}'.format(y_train))
print('x_test shape: {}'.format(x_test.shape))

x_train : [[0.1882353  0.19215687 0.1764706  ... 0.18431373 0.18039216 0.18039216]
 [0.23529412 0.23529412 0.24313726 ... 0.1254902  0.13333334 0.13333334]
 [0.15294118 0.17254902 0.20784314 ... 0.11372549 0.10196079 0.11372549]
 ...
 [0.44705883 0.45882353 0.44705883 ... 0.38431373 0.3764706  0.38431373]
 [0.4117647  0.4117647  0.41960785 ... 0.21176471 0.18431373 0.16078432]
 [0.45490196 0.44705883 0.45882353 ... 0.37254903 0.39215687 0.39607844]]
Y-train shape: [ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1
  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3
  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7
  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13
 14 14 14 14 14 14 1

# Split dataset

In [0]:
x_train, x_valid, y_train, y_valid= train_test_split(
    x_train, y_train, test_size=.05, random_state=1234,)

In [10]:
im_rows=112
im_cols=92
batch_size=512
im_shape=(im_rows, im_cols, 1)

#change the size of images
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
x_valid = x_valid.reshape(x_valid.shape[0], *im_shape)

print('x_train shape: {}'.format(y_train.shape[0]))
print('x_test shape: {}'.format(y_test.shape))

x_train shape: 216
x_test shape: (160,)


# Construction du modèle

In [0]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import Convolution2D, MaxPooling2D,MaxPool2D
import os
import numpy as np


my_model= Sequential()
my_model.add(Convolution2D(32, (3, 3), input_shape=(112,92, 1), padding='same', activation='relu'))
my_model.add(Convolution2D(32, (3, 3),activation='relu'))
my_model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

my_model.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
my_model.add(Convolution2D(64, (3, 3),activation='relu'))
my_model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

my_model.add(Flatten())

my_model.add(Dense(2024, activation='relu'))
Dropout(0.5)
my_model.add(Dense(1024,activation='relu'))
Dropout(0.5)
my_model.add(Dense(512,activation='relu'))
Dropout(0.5)
my_model.add(Dense(20,activation='softmax'))

my_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)


# Data augmentation

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False, 
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,  
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)


datagen.fit(x_train)

# L'entrainement du modèle

In [18]:
history = my_model.fit_generator(datagen.flow(np.array(x_train), np.array(y_train), batch_size=512),
                              epochs=221, verbose=2,
    validation_data=(np.array(x_valid),np.array(y_valid)))

Epoch 1/221
 - 12s - loss: 2.1331 - accuracy: 0.4444 - val_loss: 2.0145 - val_accuracy: 0.5000
Epoch 2/221
 - 12s - loss: 2.0901 - accuracy: 0.4028 - val_loss: 1.8954 - val_accuracy: 0.5833
Epoch 3/221
 - 12s - loss: 2.0084 - accuracy: 0.4537 - val_loss: 1.8313 - val_accuracy: 0.6667
Epoch 4/221
 - 12s - loss: 1.9052 - accuracy: 0.5185 - val_loss: 1.7128 - val_accuracy: 0.5833
Epoch 5/221
 - 12s - loss: 1.7893 - accuracy: 0.4954 - val_loss: 1.7677 - val_accuracy: 0.5833
Epoch 6/221
 - 12s - loss: 1.7891 - accuracy: 0.5139 - val_loss: 1.6261 - val_accuracy: 0.6667
Epoch 7/221
 - 12s - loss: 1.7343 - accuracy: 0.5417 - val_loss: 1.4607 - val_accuracy: 0.6667
Epoch 8/221
 - 12s - loss: 1.6896 - accuracy: 0.5185 - val_loss: 1.4004 - val_accuracy: 0.8333
Epoch 9/221
 - 12s - loss: 1.6800 - accuracy: 0.5417 - val_loss: 1.3991 - val_accuracy: 0.6667
Epoch 10/221
 - 12s - loss: 1.5976 - accuracy: 0.5509 - val_loss: 1.3257 - val_accuracy: 0.7500
Epoch 11/221
 - 12s - loss: 1.5787 - accuracy: 0.

# Evaluation du modèle

In [19]:
scor = my_model.evaluate( np.array(x_test),  np.array(y_test), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

test los 0.0658
test acc 0.9750


In [20]:
predicted =np.array( my_model.predict(x_test))
ynew = my_model.predict_classes(x_test)

Acc=accuracy_score(y_test, ynew)
print("accuracy : ",Acc)

accuracy :  0.975
